In [1]:
#Installation to use tensor flow
#!pip install tensor flow
#pip install Pillow

# CCN 

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
#Create a sequential model for stack layers
model = Sequential()

#Conv2d one of our convloution layer which takes 32 kernals
#The size of the input images are 64x64 using RGB 
#The activation function ReLU (Rectified Linear Unit)
model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#MaxPooling2D downsamples our input images to reduce dimensions.
#Then is put into a pool size of 2x2 for max window size. 
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten()) #Flattening the input to make a multidimensional input
model.add(Dense(128, activation='relu')) #Creates Dense layer and the number of neurons 128
model.add(Dense(3, activation='softmax'))  # 3 classes: cat, dog, 

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 31, 31, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 14, 14, 64)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 12544)             0         
                                                                 
 dense (Dense)               (None, 128)               1

# Training

In [3]:
#Compiling the model to monitor the training of the accuracy
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [2]:
#Creation of the images
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#Parameters for the images like rescaling and shearing
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
#path to training set 
training_set = train_datagen.flow_from_directory('afhq/train', target_size=(64, 64), batch_size=32, class_mode='categorical')

Found 14630 images belonging to 3 classes.


In [5]:
#Training the model, Reduced the epochs from 25 to 11 for quicker run time
model.fit(training_set, epochs=11)

Epoch 1/11
458/458 [==============================] - 51s 110ms/step - loss: 0.4301 - accuracy: 0.8256
Epoch 2/11
458/458 [==============================] - 48s 105ms/step - loss: 0.2063 - accuracy: 0.9249
Epoch 3/11
458/458 [==============================] - 49s 107ms/step - loss: 0.1631 - accuracy: 0.9393
Epoch 4/11
458/458 [==============================] - 50s 110ms/step - loss: 0.1429 - accuracy: 0.9479
Epoch 5/11
458/458 [==============================] - 50s 110ms/step - loss: 0.1208 - accuracy: 0.9557
Epoch 6/11
458/458 [==============================] - 49s 107ms/step - loss: 0.1133 - accuracy: 0.9592
Epoch 7/11
458/458 [==============================] - 49s 107ms/step - loss: 0.1011 - accuracy: 0.9634
Epoch 8/11
458/458 [==============================] - 50s 109ms/step - loss: 0.0939 - accuracy: 0.9658
Epoch 9/11
458/458 [==============================] - 50s 109ms/step - loss: 0.0832 - accuracy: 0.9712
Epoch 10/11
458/458 [==============================] - 49s 108ms/step - l

In [6]:
#For loading the our model in a keras file
model.save('my_animal_model.keras')

# Testing
## User Interface

In [ ]:
#tkinter is the GUI
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import random

# Load the trained model
loaded_model = load_model('my_animal_model.keras')

# The animal classes
classes = ['Cat', 'Dog', 'Wildlife']

# Function to make predictions
def predict_animal():
    # Get user input
    user_input = entry.get().lower()
    #Valid input
    animal_options = ['cat', 'dog', 'wild']
    # Check if the user input is valid
    if user_input not in animal_options:
        result_label.config(text="Invalid input. Please enter 'cat', 'dog', or 'wild'")
        return

    # Generate a random image number for the images from our dataset
    animal_img_number = random.randint(1, 500)

    # The image path to our test dataset 
    img_path = f'afhq/test/{user_input}/{user_input} ({animal_img_number}).jpg'

    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(64, 64))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    # Make predictions
    predictions = loaded_model.predict(img_array)

    # Get the predicted class label
    predicted_class = classes[np.argmax(predictions)]

    # Display the image
    img = Image.open(img_path)
    img = img.resize((300, 300), Image.LANCZOS)
    img = ImageTk.PhotoImage(img)
    image_label.config(image=img)
    image_label.image = img

    # The result label updated with user input
    result_label.config(text=f'The Animal Prediction: {predicted_class}')

# The main window
window = tk.Tk()
window.title("Animal Prediction AI")

# UI components
entry_label = tk.Label(window, text="Enter animal (cat, dog, wild):")
entry = tk.Entry(window)
predict_button = tk.Button(window, text="ENTER", command=predict_animal)
result_label = tk.Label(window, text="")
image_label = tk.Label(window)

# UI pack components
#Applied padding for seperation
entry_label.pack(padx=5, pady=5)
entry.pack(padx=5, pady=5)
predict_button.pack(padx=5, pady=5)
result_label.pack(padx=5, pady=5)
image_label.pack()

# Run the GUI
window.mainloop()


1/1 [==============================] - 0s 24ms/step
